# New section

In [105]:
import pandas as pd
!pip install surprise
# import surprise

from sklearn.metrics.pairwise import cosine_similarity
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset, SVD, model_selection
!pip install flask==0.12.2
!pip install flask_ngrok 


ratings=pd.read_csv('/content/drive/My Drive/datasets/Ratings .csv')
movies=pd.read_csv('/content/drive/My Drive/datasets/movies.csv')
userdata=pd.read_csv('/content/drive/My Drive/datasets/user_data.csv')
d=pd.merge(userdata,ratings,on='UserID',how='left')
df=pd.merge(d,movies,on='MovieID',how='left')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New section

***User Based Surprise SVD***

In [104]:
e = pd.read_csv('/content/drive/My Drive/datasets/svd_p.csv')

In [106]:
def recommend(userid_, n):
  f=e.loc[e[(e.uid==userid_)].est.sort_values(ascending=False).index][:n].iid.tolist()
  p=movies.loc[movies[movies.MovieID.isin(f)].index]
  k = ratings.loc[ratings[(ratings.UserID==userid_)].index]['MovieID'].tolist()
  return p.loc[p[~p.MovieID.isin(k)].index]

In [107]:
# recommend(1, 15)
def rec(a,b):
  t = recommend(a,b)[['MovieID','Title','Genres']]
  t1,t2,t3 = t['MovieID'].to_list(),t['Title'].to_list(),t['Genres'].to_list()
  return t1,t2,t3

In [108]:
rec(1,15)

([318, 593, 903, 904, 920, 953, 1198, 1234, 1250, 1272, 1307, 2905, 3198],
 ['Shawshank Redemption, The (1994)',
  'Silence of the Lambs, The (1991)',
  'Vertigo (1958)',
  'Rear Window (1954)',
  'Gone with the Wind (1939)',
  "It's a Wonderful Life (1946)",
  'Raiders of the Lost Ark (1981)',
  'Sting, The (1973)',
  'Bridge on the River Kwai, The (1957)',
  'Patton (1970)',
  'When Harry Met Sally... (1989)',
  'Sanjuro (1962)',
  'Papillon (1973)'],
 ['Drama',
  'Drama|Thriller',
  'Mystery|Thriller',
  'Mystery|Thriller',
  'Drama|Romance|War',
  'Drama',
  'Action|Adventure',
  'Comedy|Crime',
  'Drama|War',
  'Drama|War',
  'Comedy|Romance',
  'Action|Adventure',
  'Drama'])

f# New section

***Content Based (Genres)***

In [84]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer


In [86]:
movies['Genres'] = movies['Genres'].str.split('|')
movies['Genres'] = movies['Genres'].fillna("").astype('str')
movies.head(50)

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0)
matrix = tf.fit_transform(movies['Genres'])
csim = linear_kernel(matrix,matrix)


titles = movies['Title']
indices = pd.Series(movies.index, index=movies['Title'])

def c_rec(title):
    idx = indices[title]
    sim_sc = list(enumerate(csim[idx]))
    sim_sc = sorted(sim_sc, key=lambda x: x[1], reverse=True)[1:]
    movie_idx = [i[0] for i in sim_sc]
    print('Movies Similar to : ',title)
    yy=pd.DataFrame(movies.Title.iloc[movie_idx])[:10]
    return yy

In [101]:
c_rec('Dracula: Dead and Loving It (1995)')

Movies Similar to :  Dracula: Dead and Loving It (1995)


,Title
326,Tales from the Hood (1995)
726,Cemetery Man (Dellamorte Dellamore) (1994)
789,"Frighteners, The (1996)"
1221,Braindead (1992)
1235,Bad Taste (1987)
1258,Young Frankenstein (1974)
1310,April Fool's Day (1986)
1409,"Machine, The (1994)"
1934,Gremlins (1984)
1935,Gremlins 2: The New Batch (1990)


***CountVectorizer***

In [81]:
df2 = pd.read_csv('/content/drive/My Drive/datasets/movies.csv')

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['Genres'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['Title'])
all_titles = [df2['Title'][i] for i in range(len(df2['Title']))]

def get_recommendations(title):
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    tit = df2['Title'].iloc[movie_indices]
    dat = df2['Genres'].iloc[movie_indices]
    return_df = pd.DataFrame(columns=['Title','Genres'])
    return_df['Title'] = tit
    return_df['Genres'] = dat
    return return_df

In [89]:
get_recommendations('Paths of Glory (1957)')

,Title,Genres
153,Beyond Rangoon (1995),Drama|War
332,"Walking Dead, The (1995)",Drama|War
523,Schindler's List (1993),Drama|War
641,Courage Under Fire (1996),Drama|War
967,Nothing Personal (1995),Drama|War
979,Michael Collins (1996),Drama|War
1074,Platoon (1986),Drama|War
1162,Paths of Glory (1957),Drama|War
1190,Apocalypse Now (1979),Drama|War
1199,Ran (1985),Drama|War


In [109]:
import flask
import difflib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask,request,render_template
from flask_ngrok import run_with_ngrok


app = flask.Flask(__name__, template_folder='/content/drive/My Drive/datasets/templates')

run_with_ngrok(app)



@app.route('/', methods=['GET', 'POST'])
def index(): return render_template('index.html')


@app.route("/predict/",methods=["POST"])
def main():
  if request.method == 'POST': m_name = request.form['movie_name'].title()
  if request.method == "GET" : m_name = request.args.get('movie_name').title()
  

  print(m_name)
  # check = difflib.get_close_matches(m_name,all_titles,cutout=0.50,n=1)
  if m_name not in all_titles: return(render_template('index.html'))
  else:
    result_final = get_recommendations(m_name)
    names = []
    dates = []
    for i in range(len(result_final)):
      names.append(result_final.iloc[i][0])
      dates.append(result_final.iloc[i][1])
    return flask.render_template('positive.html',result = 1,ret = zip(names,dates),search_name=m_name)


@app.route("/user/",methods=["POST"])
def userPred():
  print(request.method)



  try: a = int(request.form['user_id'])
  except: return render_template('index.html')
  # b = request.form['recommendation']
  print(a)
  b = 10
  t1,t2,t3 = rec(a,b)
  return render_template('positive.html',neut = 1,ret = zip(t1,t2,t3))

if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://16031e9306a9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Jul/2020 10:50:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2020 10:50:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Jul/2020 10:50:40] "POST /predict/ HTTP/1.1" 200 -


Toy Story (1995)


127.0.0.1 - - [19/Jul/2020 10:50:49] "POST /user/ HTTP/1.1" 200 -


POST
555
